# Demonstrating different sizing modes with H2 electrolyzer

This notebook shows how to use different sizing modes (`size_from_config`, `resize_for_max_feedstock`, and `resize_for_max_product`) using a system with a HOPP hybrid electricity plant, a PEM hydrogen electrolyzer, and a synloop ammonia model.
The electrolyzer is sized differently depending on the `size_mode` option, the feedstock/product identified by `resize_by_flow`, and the conencted tech identified by `resize_by_tech`.
If `iterative_mode` is enabled, the problem will attempt to use a connected variable for the `resize_for_max_product` mode that creates a feedback loop, this is not working yet (crashes at `pf.solve_price()`).

## Sizing modes

For the converters involved in this example, three different parameters have been defined, which determine how the `PerformanceModel.compute()` function will resize the capacity.
There is an additional new parameter (which is intended to be temporary): `iterative_mode`, which switches between two methods of implementing the `resize_for_max_product` mode.
This parameter will be removed when we decide which method to use.

- `size_from_config`: In this mode, converters function as they always have previously:
    - The size of the asset is fixed within the `compute()` function, and is read from the config.
- `resize_for_max_feedstock`: In this mode, the size of the asset is adjusted to be able to utilize all of the available feedstock:
    - The size of the asset is *calculated* within the `compute()` function, by how big it needs to be to utilize all of the `<feedstock>_in`.
- `resize_for_max_product`: In this mode, the size of the asset is adjusted to be able to supply its product to the full capacity of another downstream converter. This is done two different ways
    - The size of the asset is calculated within the `compute()` function, by how big it needs to be to match the capacity of a connected downstream tech.
    - If `iterative_mode` is False, the asset will import a method from the downstream tech to determine how much `<product>_out` is needed to match the downstream tech's max capacity.
      - This requies the technology modules to have access to the *whole* tech_config, not just the parameters for their specific technology. Edits have been made to core code to allow this.
    - If `iterative_mode` is True, the asset will use a connected variable `max_<product>_capacity` from the downstream tech. This creates a feedback loop.
      - This mode is not yet working, because `pf.solve_price()` will not run - more changes need to be made to allow ProFast to run separately from the feedback loop.
     

## Example plant setup

Here, there are four technologies in the the `tech_config.yaml`: An `electrolyzer`, a `hopp` (configured for electricity), `h2_storage` and `ammonia` (to consume the hydrogen).

### Example technology_interconnections within plant_config.yaml
```yaml
technology_interconnections: [
  ["hopp","electrolyzer","electricity","cable"],
  ["electrolyzer", "h2_storage", "efficiency"],
  ["electrolyzer", "h2_storage", "hydrogen", "pipe"],
  ["h2_storage", "ammonia", "hydrogen", "pipe"],
  ["ammonia", "electrolyzer", "max_hydrogen_capacity"],
]
```
The final connection from the ammonia back to the electroyzer is used in `iterative_mode` for `resize_for_max_product` to size the electrolyzer.

### Example tech_config.yaml (most parameters skipped for brevity)
```yaml
technologies:
  electrolyzer:
    performance_model:
      model: "eco_pem_electrolyzer_performance"
    cost_model:
      model: "singlitico_electrolyzer_cost"
    model_inputs:
      performance_parameters:
        size_mode: "resize_for_max_product"
        resize_by_flow: "hydrogen"
        resize_by_tech: "ammonia"
        iterative_mode: True
        n_clusters: 16
        cluster_rating_MW: 40
  hopp:
    electrolyzer_rating: 640. # MW
  h2_storage:
    model_inputs:
      performance_parameters:
        rating: 640
  ammonia:
    model_inputs:
      performance_parameters:
        iterative_mode: True
```

## Running an example

### `size_from_config` mode

The example `.yaml` files have been set up in `size_from_config` mode.
In this case the electrolyzer has been sized to 640 MW (same as previous example), but the electricity profile going in only has a max of 600 MW.
Thus, the LCOH ($5.85/kg H2) and LCOA ($1.10/kg NH3) not optimal - the electrolyzer is too small to utilize all of the power and too big for the ammonia plant to utilize all the hydrogen.

In [1]:
from h2integrate.core.h2integrate_model import H2IntegrateModel


# Create a H2Integrate model
model = H2IntegrateModel("14_size_mode_config.yaml")

# Run the model
model.run()

model.post_process()

c:\Users\jmartin4\Documents\Code\jmartin4nrel\H2Integrate\examples\14_sizing_modes\log\hybrid_systems_2025-08-22T11.29.32.970574.log
62 Input(s) in 'model'

varname                               val                   units      prom_name                                                
------------------------------------  --------------------  ---------  ---------------------------------------------------------
plant
  hopp
    hopp
      wind_turbine_rating_kw          [6000.]               kW         hopp.wind_turbine_rating_kw                              
      pv_capacity_kw                  [400000.]             kW         hopp.pv_capacity_kw                                      
      battery_capacity_kw             [96.]                 kW         hopp.battery_capacity_kw                                 
      battery_capacity_kwh            [96.]                 kW*h       hopp.battery_capacity_kwh                                
  hopp_to_electrolyzer_cable
    electricity_in

### `resize_by_max_feedstock` mode

In this case, the electrolyzer will be sized to match the maximum `electricity_in` coming from HOPP
This increases the electrolyzer size to 1120 MW, the closest multiple of 40 MW (the cluster size) matching the max HOPP power output of 1103 MW.
This reduces the LCOH to $5.66/kg H2, but increases the LCOA to $1.30/kg NH3, since electrolyzer size was matched to HOPP but not ammonia produciton.

In [2]:
# Create a H2Integrate model
feed_model = H2IntegrateModel("14_size_mode_feedstock.yaml")

# Run the model
feed_model.run()

feed_model.post_process()

62 Input(s) in 'model'

varname                               val                   units      prom_name                                                
------------------------------------  --------------------  ---------  ---------------------------------------------------------
plant
  hopp
    hopp
      wind_turbine_rating_kw          [6000.]               kW         hopp.wind_turbine_rating_kw                              
      pv_capacity_kw                  [400000.]             kW         hopp.pv_capacity_kw                                      
      battery_capacity_kw             [96.]                 kW         hopp.battery_capacity_kw                                 
      battery_capacity_kwh            [96.]                 kW*h       hopp.battery_capacity_kwh                                
  hopp_to_electrolyzer_cable
    electricity_in                    |60512729.77529646|   kW         hopp_to_electrolyzer_cable.electricity_in                
  electrolyzer
    eco

### `resize_by_max_feedstock` mode: non-iterative

In this case, the electrolyzer will be sized to match the maximum `hydrogen_in` used by ammonia production
This decreases the electrolyzer size to 280 MW, the closest multiple of 40 MW (the cluster size) that will ensure an h2 produciton capacity that matches the ammonia plant's h2 intake at its max ammonia produciton capacity.
This increases the LCOH to $9.86/kg H2, but reduces the LCOA to $0.95/kg NH3, since electrolyzer size was matched to ammonia produciton but not HOPP.

In [4]:
# Create a H2Integrate model
prod_model = H2IntegrateModel("14_size_mode_product.yaml")

# Run the model
prod_model.run()

prod_model.post_process()

62 Input(s) in 'model'

varname                               val                   units      prom_name                                                
------------------------------------  --------------------  ---------  ---------------------------------------------------------
plant
  hopp
    hopp
      wind_turbine_rating_kw          [6000.]               kW         hopp.wind_turbine_rating_kw                              
      pv_capacity_kw                  [400000.]             kW         hopp.pv_capacity_kw                                      
      battery_capacity_kw             [96.]                 kW         hopp.battery_capacity_kw                                 
      battery_capacity_kwh            [96.]                 kW*h       hopp.battery_capacity_kwh                                
  hopp_to_electrolyzer_cable
    electricity_in                    |60512729.77529646|   kW         hopp_to_electrolyzer_cable.electricity_in                
  electrolyzer
    eco